In [1]:
import matplotlib.pyplot as plt
import csv
import pandas as pd
import operator
from statistics import mean,pstdev
import numpy as np
import random
import copy

In [2]:
path="ml-latest-small/"
ratings_path=path+"ratings.csv"
movies_path=path+"movies.csv"

In [3]:
ratings_df=pd.read_csv(ratings_path)
ratings_df.head()


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
movie_indices=[]
inverse_movie_map={}
movies_df=pd.read_csv(movies_path)
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
for i in range(len(movies_df)):
    movie_indices.append(movies_df['movieId'][i])
    inverse_movie_map[movies_df['movieId'][i]]=i

In [6]:
number_of_items=len(movie_indices)
number_of_users={}
for i in range(len(ratings_df)):
    number_of_users[ratings_df['userId'][i]]=1

In [7]:
number_of_users=len(number_of_users)
user_sim_matrix_cosine=np.load(path+'user_sim_matrix_cosine.npy')
user_sim_matrix_pearson=np.load(path+'user_sim_matrix_pearson.npy')
ratings_matrix_train=np.load(path+'ratings_matrix_train.npy')
train_nos=np.load(path+'train_nos.npy')
test_nos=np.load(path+'test_nos.npy')
ratings_matrix_train_mean_centred=np.load(path+'ratings_matrix_train_mean_centred.npy')

In [8]:
correct_predictions=[]
for i in test_nos:
    correct_predictions.append(ratings_df['rating'][i])

In [9]:
def cosine_sim(a,b):
    mean_a=mean_with_nan(a)
    mean_b=mean_with_nan(b)
    #print(mean_a)
    #print(mean_b)
    num=0.0
    den_left=0.0
    den_right=0.0
    for i in range(len(a)):
        if (not np.isnan(a[i])) and (not np.isnan(b[i])):
            num+=(a[i])*(b[i])
            den_left+=(a[i])*(a[i])
            den_right+=(b[i])*(b[i])
    den_left=np.sqrt(den_left)
    #print(den_left)
    den_right=np.sqrt(den_right)
    #print(den_right)
    if (den_right==0 or den_left==0):
        return -1
    return num/((den_left)*(den_right))

In [10]:
def mean_absolute_error(a,b):
    error_sum=0.0
    for i in range(len(a)):
        error_sum+=abs(a[i]-b[i])
    error_sum/=len(a)
    return error_sum     

In [11]:
def rmse_error(a,b):
    error_sum=0.0
    for i in range(len(a)):
        error_sum+=(a[i]-b[i])*(a[i]-b[i])
    error_sum/=len(a)
    return np.sqrt(error_sum) 

In [12]:
def mean_with_nan(list_a):
    count=0
    sum=0.0
    for i in list_a:
        if not np.isnan(i):
            #print(i)
            count+=1
            sum+=i
    #print(sum) 
    if(count==0):
        return 0
    return sum/count        
    

In [13]:
mean_of_users=[]
for i in range(number_of_users):
    mean_of_users.append(mean_with_nan(ratings_matrix_train[i]))
mean_of_items=[]
for i in range(number_of_items):
    mean_of_items.append(mean_with_nan(ratings_matrix_train[:,i]))    
    

In [14]:
def pearson_coff(a,b):
    mean_a=mean_with_nan(a)
    mean_b=mean_with_nan(b)
    #print(mean_a)
    #print(mean_b)
    num=0.0
    den_left=0.0
    den_right=0.0
    for i in range(len(a)):
        if (not np.isnan(a[i])) and (not np.isnan(b[i])):
            num+=(a[i]-mean_a)*(b[i]-mean_b)
            den_left+=(a[i]-mean_a)*(a[i]-mean_a)
            den_right+=(b[i]-mean_b)*(b[i]-mean_b)
    den_left=np.sqrt(den_left)
    #print(den_left)
    den_right=np.sqrt(den_right)
    #print(den_right)
    if (den_right==0 or den_left==0):
        return -1
    return num/((den_left)*(den_right))

In [15]:
def mean_list(a,b):
    c=[]
    for i in range(len(a)):
        c.append((a[i]+b[i])/2)
    return c    

In [38]:
k=10
cosine_result=[]
for i in test_nos:
    movie_no=inverse_movie_map[ratings_df['movieId'][i]]
    user_no=ratings_df['userId'][i]-1
    
    top_k_similar_users_same_item_rated=[]
    for j in range(0,number_of_users):
        if(j!=user_no):
            if(not np.isnan(ratings_matrix_train[j][movie_no])):
                top_k_similar_users_same_item_rated.append((user_sim_matrix_cosine[user_no][j],ratings_matrix_train[j][movie_no]-mean_of_users[j]))
    #print(len(top_k_similar_users_same_item_rated))
    #print(top_k_similar_users_same_item_rated)            
    top_k_similar_users_same_item_rated=sorted(top_k_similar_users_same_item_rated,key=lambda x: x[0],reverse=True)  
    num=0.0
    den=0.0
    if(not len(top_k_similar_users_same_item_rated)<k):
        for p in range(0,k):
            num+=top_k_similar_users_same_item_rated[p][0]*top_k_similar_users_same_item_rated[p][1]
            den+=abs(top_k_similar_users_same_item_rated[p][0])
        cosine_result.append(mean_of_users[user_no]+(num/den)) 
    elif(len(top_k_similar_users_same_item_rated)>0):
        for p in range(0,len(top_k_similar_users_same_item_rated)):
            num+=top_k_similar_users_same_item_rated[p][0]*top_k_similar_users_same_item_rated[p][1]
            den+=abs(top_k_similar_users_same_item_rated[p][0])
            
        cosine_result.append(mean_of_users[user_no]+(num/den)) 
    else:
        cosine_result.append(mean_with_nan(ratings_matrix_train[user_no]))


In [17]:
k=10
cosine_item_result=[]
for i in test_nos:
    movie_no=inverse_movie_map[ratings_df['movieId'][i]]
    user_no=ratings_df['userId'][i]-1
    
    top_k_similar_items_same_users_rated=[]
    for j in range(0,number_of_items):
        if(j!=movie_no):
            if(not np.isnan(ratings_matrix_train[user_no][j])):
                top_k_similar_items_same_users_rated.append((item_sim_matrix_cosine[movie_no][j],ratings_matrix_train[user_no][j]-mean_of_items[j]))
    #print(len(top_k_similar_users_same_item_rated))
    #print(top_k_similar_users_same_item_rated)            
    top_k_similar_items_same_users_rated=sorted(top_k_similar_items_same_users_rated,key=lambda x: x[0],reverse=True)  
    num=0.0
    den=0.0
    if(not len(top_k_similar_items_same_users_rated)<k):
        for p in range(0,k):
            num+=top_k_similar_items_same_users_rated[p][0]*top_k_similar_items_same_users_rated[p][1]
            den+=abs(top_k_similar_items_same_users_rated[p][0])
        cosine_item_result.append(mean_of_items[movie_no]+(num/den)) 
    elif(len(top_k_similar_items_same_users_rated)>0):
        for p in range(0,len(top_k_similar_items_same_users_rated)):
            num+=top_k_similar_items_same_users_rated[p][0]*top_k_similar_items_same_users_rated[p][1]
            den+=abs(top_k_similar_items_same_users_rated[p][0])
            
        cosine_item_result.append(mean_of_items[item_no]+(num/den)) 
    else:
        cosine_item_result.append(mean_with_nan(ratings_matrix_train[:,movie_no]))


In [ ]:
k=10
cosine_item_raw_result=[]
for i in test_nos:
    movie_no=inverse_movie_map[ratings_df['movieId'][i]]
    user_no=ratings_df['userId'][i]-1
    
    top_k_similar_items_same_users_rated=[]
    for j in range(0,number_of_items):
        if(j!=movie_no):
            if(not np.isnan(ratings_matrix_train[user_no][j])):
                top_k_similar_items_same_users_rated.append((item_sim_matrix_cosine[movie_no][j],ratings_matrix_train[user_no][j]))
    #print(len(top_k_similar_users_same_item_rated))
    #print(top_k_similar_users_same_item_rated)            
    top_k_similar_items_same_users_rated=sorted(top_k_similar_items_same_users_rated,key=lambda x: x[0],reverse=True)  
    num=0.0
    den=0.0
    if(not len(top_k_similar_items_same_users_rated)<k):
        for p in range(0,k):
            num+=top_k_similar_items_same_users_rated[p][0]*top_k_similar_items_same_users_rated[p][1]
            den+=abs(top_k_similar_items_same_users_rated[p][0])
        cosine_item_raw_result.append((num/den)) 
    elif(len(top_k_similar_items_same_users_rated)>0):
        for p in range(0,len(top_k_similar_items_same_users_rated)):
            num+=top_k_similar_items_same_users_rated[p][0]*top_k_similar_items_same_users_rated[p][1]
            den+=abs(top_k_similar_items_same_users_rated[p][0])
            
        cosine_item_raw_result.append((num/den)) 
    else:
        cosine_item_raw_result.append(mean_with_nan(ratings_matrix_train[:,movie_no]))


In [37]:
k=10
pearson_result=[]
for i in test_nos:
    movie_no=inverse_movie_map[ratings_df['movieId'][i]]
    user_no=ratings_df['userId'][i]-1
    
    top_k_similar_users_same_item_rated=[]
    for j in range(0,number_of_users):
        if(j!=user_no):
            if(not np.isnan(ratings_matrix_train[j][movie_no])):
                top_k_similar_users_same_item_rated.append((user_sim_matrix_pearson[user_no][j],ratings_matrix_train[j][movie_no]-mean_of_users[j]))
    #print(len(top_k_similar_users_same_item_rated))
    #print(top_k_similar_users_same_item_rated)            
    top_k_similar_users_same_item_rated=sorted(top_k_similar_users_same_item_rated,key=lambda x: x[0],reverse=True)  
    num=0.0
    den=0.0
    if(not len(top_k_similar_users_same_item_rated)<k):
        for p in range(0,k):
            num+=top_k_similar_users_same_item_rated[p][0]*top_k_similar_users_same_item_rated[p][1]
            den+=abs(top_k_similar_users_same_item_rated[p][0])
        pearson_result.append(mean_of_users[user_no]+(num/den)) 
    elif(len(top_k_similar_users_same_item_rated)>0):
        for p in range(0,len(top_k_similar_users_same_item_rated)):
            num+=top_k_similar_users_same_item_rated[p][0]*top_k_similar_users_same_item_rated[p][1]
            den+=abs(top_k_similar_users_same_item_rated[p][0])
            
        pearson_result.append(mean_of_users[user_no]+(num/den)) 
    else:
        pearson_result.append(mean_with_nan(ratings_matrix_train[user_no]))


In [19]:
k=10
pearson_item_result=[]
for i in test_nos:
    movie_no=inverse_movie_map[ratings_df['movieId'][i]]
    user_no=ratings_df['userId'][i]-1
    
    top_k_similar_items_same_users_rated=[]
    for j in range(0,number_of_items):
        if(j!=movie_no):
            if(not np.isnan(ratings_matrix_train[user_no][j])):
                top_k_similar_items_same_users_rated.append((item_sim_matrix_pearson[movie_no][j],ratings_matrix_train[user_no][j]-mean_of_items[j]))
    #print(len(top_k_similar_users_same_item_rated))
    #print(top_k_similar_users_same_item_rated)            
    top_k_similar_items_same_users_rated=sorted(top_k_similar_items_same_users_rated,key=lambda x: x[0],reverse=True)  
    num=0.0
    den=0.0
    if(not len(top_k_similar_items_same_users_rated)<k):
        for p in range(0,k):
            num+=top_k_similar_items_same_users_rated[p][0]*top_k_similar_items_same_users_rated[p][1]
            den+=abs(top_k_similar_items_same_users_rated[p][0])
        pearson_item_result.append(mean_of_items[movie_no]+(num/den)) 
    elif(len(top_k_similar_items_same_users_rated)>0):
        for p in range(0,len(top_k_similar_items_same_users_rated)):
            num+=top_k_similar_items_same_users_rated[p][0]*top_k_similar_items_same_users_rated[p][1]
            den+=abs(top_k_similar_items_same_users_rated[p][0])
            
        pearson_item_result.append(mean_of_items[item_no]+(num/den)) 
    else:
        pearson_item_result.append(mean_with_nan(ratings_matrix_train[:,movie_no]))


In [ ]:
# k=10
# pearson_item_raw_result=[]
# for i in test_nos:
#     movie_no=inverse_movie_map[ratings_df['movieId'][i]]
#     user_no=ratings_df['userId'][i]-1
    
#     top_k_similar_items_same_users_rated=[]
#     for j in range(0,number_of_items):
#         if(j!=movie_no):
#             if(not np.isnan(ratings_matrix_train[user_no][j])):
#                 top_k_similar_items_same_users_rated.append((item_sim_matrix_pearson[movie_no][j],ratings_matrix_train[user_no][j]))
#     #print(len(top_k_similar_users_same_item_rated))
#     #print(top_k_similar_users_same_item_rated)            
#     top_k_similar_items_same_users_rated=sorted(top_k_similar_items_same_users_rated,key=lambda x: x[0],reverse=True)  
#     num=0.0
#     den=0.0
#     if(not len(top_k_similar_items_same_users_rated)<k):
#         for p in range(0,k):
#             num+=top_k_similar_items_same_users_rated[p][0]*top_k_similar_items_same_users_rated[p][1]
#             den+=abs(top_k_similar_items_same_users_rated[p][0])
#         pearson_item_raw_result.append((num/den)) 
#     elif(len(top_k_similar_items_same_users_rated)>0):
#         for p in range(0,len(top_k_similar_items_same_users_rated)):
#             num+=top_k_similar_items_same_users_rated[p][0]*top_k_similar_items_same_users_rated[p][1]
#             den+=abs(top_k_similar_items_same_users_rated[p][0])
            
#         pearson_item_raw_result.append((num/den)) 
#     else:
#         pearson_item_raw_result.append(mean_with_nan(ratings_matrix_train[:,movie_no]))


In [16]:
k=10
pearson_raw_result=[]
for i in test_nos:
    movie_no=inverse_movie_map[ratings_df['movieId'][i]]
    user_no=ratings_df['userId'][i]-1
    
    top_k_similar_users_same_item_rated=[]
    for j in range(0,number_of_users):
        if(j!=user_no):
            if(not np.isnan(ratings_matrix_train[j][movie_no])):
                top_k_similar_users_same_item_rated.append((user_sim_matrix_pearson[user_no][j],ratings_matrix_train[j][movie_no]))
    #print(len(top_k_similar_users_same_item_rated))
    #print(top_k_similar_users_same_item_rated)            
    top_k_similar_users_same_item_rated=sorted(top_k_similar_users_same_item_rated,key=lambda x: x[0],reverse=True)  
    num=0.0
    den=0.0
    if(not len(top_k_similar_users_same_item_rated)<k):
        for p in range(0,k):
            num+=top_k_similar_users_same_item_rated[p][0]*top_k_similar_users_same_item_rated[p][1]
            den+=abs(top_k_similar_users_same_item_rated[p][0])
        pearson_raw_result.append(num/den) 
    elif(len(top_k_similar_users_same_item_rated)>0):
        for p in range(0,len(top_k_similar_users_same_item_rated)):
            num+=top_k_similar_users_same_item_rated[p][0]*top_k_similar_users_same_item_rated[p][1]
            den+=abs(top_k_similar_users_same_item_rated[p][0])
        pearson_raw_result.append(num/den) 
    else:
        pearson_raw_result.append(mean_with_nan(ratings_matrix_train[user_no]))


In [17]:
k=10
cosine_raw_result=[]
for i in test_nos:
    movie_no=inverse_movie_map[ratings_df['movieId'][i]]
    user_no=ratings_df['userId'][i]-1
    
    top_k_similar_users_same_item_rated=[]
    for j in range(0,number_of_users):
        if(j!=user_no):
            if(not np.isnan(ratings_matrix_train[j][movie_no])):
                top_k_similar_users_same_item_rated.append((user_sim_matrix_cosine[user_no][j],ratings_matrix_train[j][movie_no]))
    #print(len(top_k_similar_users_same_item_rated))
    #print(top_k_similar_users_same_item_rated)            
    top_k_similar_users_same_item_rated=sorted(top_k_similar_users_same_item_rated,key=lambda x: x[0],reverse=True)  
    num=0.0
    den=0.0
    if(not len(top_k_similar_users_same_item_rated)<k):
        for p in range(0,k):
            num+=top_k_similar_users_same_item_rated[p][0]*top_k_similar_users_same_item_rated[p][1]
            den+=abs(top_k_similar_users_same_item_rated[p][0])
        cosine_raw_result.append(num/den) 
    elif(len(top_k_similar_users_same_item_rated)>0):
        for p in range(0,len(top_k_similar_users_same_item_rated)):
            num+=top_k_similar_users_same_item_rated[p][0]*top_k_similar_users_same_item_rated[p][1]
            den+=abs(top_k_similar_users_same_item_rated[p][0])
        cosine_raw_result.append(num/den) 
    else:
        cosine_raw_result.append(mean_with_nan(ratings_matrix_train[user_no]))


In [43]:
rmse_error(correct_predictions,hybrid_cosine)

0.9564611034081687

In [44]:
mean_absolute_error(correct_predictions,hybrid_cosine)

0.7277524270318928

In [39]:
hybrid_pearson=mean_list(pearson_result,pearson_item_result)
hybrid_cosine=mean_list(cosine_result,cosine_item_result)

In [ ]:
item_sim_matrix_pearson=np.zeros(shape=(number_of_items,number_of_items))
item_sim_matrix_pearson.fill(np.nan)
for i in range(0,number_of_items):
    if(i%50==0):
        print(i)
    for j in range(i,number_of_items):
        item_sim_matrix_pearson[i][j]=pearson_coff(ratings_matrix_train[:,i],ratings_matrix_train[:,j])
        item_sim_matrix_pearson[j][i]=item_sim_matrix_pearson[i][j]

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750


In [ ]:
#np.save(path+'item_sim_matrix_pearson',item_sim_matrix_pearson)

In [32]:
#del item_sim_matrix_pearson,item_sim_matrix_cosine

In [25]:
item_map={}
for i in range(number_of_items):
    for j in range(number_of_users):
        if(not np.isnan(ratings_matrix_train[j][i])):
            if i in item_map:
                item_map[i].append(j)
            else:
                item_map[i]=[]
                item_map[i].append(j)
        

KeyboardInterrupt: 

In [ ]:
weight=0.4


In [ ]:
item_sim_matrix_cosine=np.zeros(shape=(number_of_items,number_of_items))
item_sim_matrix_cosine.fill(np.nan)
for i in range(0,number_of_items):
    if(i%50==0):
        print(i)
    for j in range(i,number_of_items):
        item_sim_matrix_cosine[i][j]=cosine_sim(ratings_matrix_train[:,i],ratings_matrix_train[:,j])
        item_sim_matrix_cosine[j][i]=item_sim_matrix_cosine[i][j]

In [ ]:
#np.save(path+'item_sim_matrix_cosine',item_sim_matrix_cosine)

In [2]:
a=[1,3,8,6]

In [3]:
if 8 in a:
    print("yo")

yo
